In [1]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

In [21]:
import nltk
# nltk.download('stopwords') #run once and comment it out to avoid it downloading multiple times
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
# from nltk.stem.porter import PorterStemmer

import string
import re
# import textblob
# from textblob import TextBlob


# from wordcloud import WordCloud, STOPWORDS
from emot.emo_unicode import UNICODE_EMOJI

In [27]:
# comment it out once you have gathered your data 
# to avoid running out of the wait time gather the data per month
query = "adani"
tweets = []
for i, tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
    if i>2000:
        break
    else:
        tweets.append([tweet.date, tweet.id, tweet.url, tweet.user.username, tweet.sourceLabel, tweet.user.location, tweet.content, tweet.likeCount, tweet.retweetCount])
df = pd.DataFrame(tweets, columns = ['Date', 'ID', 'url', 'username', 'source', 'location', 'tweet', 'num_of_likes', 'num_of_retweet'])
df.to_csv('sentiment.csv', mode = 'a')

/tmp/ipykernel_9411/2832477045.py:9: FutureWarning: content is deprecated, use rawContent instead
  tweets.append([tweet.date, tweet.id, tweet.url, tweet.user.username, tweet.sourceLabel, tweet.user.location, tweet.content, tweet.likeCount, tweet.retweetCount])


In [8]:
df =  pd.read_csv('sentiment.csv', encoding = 'unicode_escape')
df.head(5)

,Unnamed: 0,Date,ID,url,username,source,location,tweet,num_of_likes,num_of_retweet
0,NaN,Date,ID,url,username,source,location,tweet,num_of_likes,num_of_retweet
1,0.0,2023-02-03 07:24:17+00:00,1621409223283462146,https://twitter.com/ayaz_karbelkar/status/1621...,ayaz_karbelkar,Twitter for Android,"Mumbai, India",@RiaRevealed Greatest respect for ace lawyer C...,0,0
2,1.0,2023-02-03 07:24:16+00:00,1621409218426470402,https://twitter.com/ShivKumarshah56/status/162...,ShivKumarshah56,Twitter for Android,India,@ActivistSandeep See screen shots i tweeted of...,0,0
3,2.0,2023-02-03 07:24:15+00:00,1621409214563504128,https://twitter.com/MypicF/status/162140921456...,MypicF,Twitter for Android,NaN,@saudev001 @ASHOKBH40280132 If business is not...,0,0
4,3.0,2023-02-03 07:24:13+00:00,1621409205419921409,https://twitter.com/VaghRv/status/162140920541...,VaghRv,Twitter for Android,NaN,@AdaniOnline @gautam_adani à¤à¤° à¤à¤ à¤¬à¤...,0,0
5,4.0,2023-02-03 07:24:13+00:00,1621409205201829889,https://twitter.com/ARPITHABRS/status/16214092...,ARPITHABRS,Twitter for iPhone,"Hyderabad, India",@KTRBRS #AcheDin started for Adani in #AmritKa...,0,0
6,5.0,2023-02-03 07:24:13+00:00,1621409204685930497,https://twitter.com/RajendraKhoja10/status/162...,RajendraKhoja10,Twitter for Android,NaN,@iTRJat Ak Adani Or puri ops barabar he jitna ...,0,0
7,6.0,2023-02-03 07:24:13+00:00,1621409204245495808,https://twitter.com/VijayEllenabad/status/1621...,VijayEllenabad,Twitter for Android,"Haryana, India",Adani Group à¤à¥ Financial Fraud à¤ªà¤° MP @...,0,0
8,7.0,2023-02-03 07:24:13+00:00,1621409203402457089,https://twitter.com/Fuhrer1418/status/16214092...,Fuhrer1418,Twitter for Android,Earth,@TrulyMonica Lesson of the Adani episode: You ...,0,0
9,8.0,2023-02-03 07:24:09+00:00,1621409189036974080,https://twitter.com/AkshayyyHere___/status/162...,AkshayyyHere___,Twitter for Android,"Madhya Pradesh, India",Adani going huge ð¯,0,0


In [16]:
eng_stop_words = list(stopwords.words('english'))

In [20]:
emoji = list(UNICODE_EMOJI.keys())

In [22]:
def ProcessedTweets(text):
    #changing tweet text to small letters
    text = text.lower()
    
    # # Removing @ and links 
    # text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", text).split())
    # # removing repeating characters
    # text = re.sub(r'\@\w+|\#\w+|\d+', '', text)
    # # removing punctuation and numbers
    # punct = str.maketrans('', '', string.punctuation+string.digits)
    # text = text.translate(punct)
    # # tokenizing words and removing stop words from the tweet text
    
    # tokens = word_tokenize(text)  
    # filtered_words = [w for w in tokens if w not in eng_stop_words]
    # filtered_words = [w for w in filtered_words if w not in emoji]
    # # lemmetizing words
    # lemmatizer = WordNetLemmatizer() 
    # lemma_words = [lemmatizer.lemmatize(w) for w in filtered_words]
    # text = " ".join(lemma_words)
    
    return text